In [1]:
import geopandas as gpd
import rasterio
import pandas as pd
from rasterio.sample import sample_gen
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,ConfusionMatrixDisplay,\
                            precision_score,recall_score,f1_score,roc_auc_score, roc_curve

# import matplotlib as plt
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier  #https://github.com/sentinel-hub/eo-learn-examples/blob/main/custom-script/machine-learning-evalscript.ipynb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)
from sklearn import svm
from sklearn.svm import LinearSVC

In [2]:
# Load the CSV data
# data = pd.read_csv('training_data_python_textures.csv')
training_data_path="output/sample/training_data_eaton_indices.csv"
training_data=pd.read_csv(training_data_path)

# get from feature_extration.ipynb
# columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre"]
# columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI"]
# columns_names= feature_column_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'] #,"ΔRVI",'Δvv_vh_ratio',"ΔRDFI","dTRAD"

columns_names=["RBD_VV", "RBD_VH", "RBR_VV","dod_RBR_VV", "RBR_VH","dod_RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI"]

# columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI"]
# columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']

# columns_names=["gamm_vh_pre","gamm_vv_pre","gamm_vh_post","gamm_vv_post","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']


In [3]:
# Separate features (SAR bands) and labels (e.g., class column)
X = training_data[columns_names]  # Features (SAR band values)
Y = training_data['class']  # Labels (add 'Class' column in your CSV if available)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [4]:
#To visualize data in X_train
pd.DataFrame(X_train)

,RBD_VV,RBD_VH,RBR_VV,dod_RBR_VV,RBR_VH,dod_RBR_VH,ΔRVI,Δvv_vh_ratio,ΔRDFI,RCBI
6590,-0.082487,0.017430,-0.196356,4.811109,0.181345,4.443387,3.061849,-3.777021,-0.264454,-0.065057
4022,0.018623,0.056777,0.048254,1.228442,0.474655,12.083760,3.276065,-4.263999,-0.344678,0.075399
5571,0.011695,0.006900,0.061689,3.142904,0.096128,4.897583,0.249904,-0.344382,-0.031576,0.018595
3308,0.003907,-0.005191,0.036023,2.830136,-0.196608,15.447451,-1.871319,2.326318,0.167959,-0.001284
2117,0.089575,-0.006922,0.220852,6.673275,-0.102345,3.092543,-2.760196,3.231975,0.184499,0.082653
...,...,...,...,...,...,...,...,...,...,...
5226,0.050299,-0.012632,0.103827,2.229199,-0.174428,3.745121,-2.418125,2.782563,0.145299,0.037667
5390,-0.013401,0.001313,-0.025162,0.405700,0.014747,0.237770,0.341913,-0.399085,-0.022556,-0.012088
860,-0.068323,0.092273,-0.096661,0.706260,0.532881,3.893585,5.063886,-6.295425,-0.444194,0.023950
7603,-0.020759,0.001091,-0.069092,1.699241,0.012403,0.305042,0.630133,-0.814952,-0.065777,-0.019668


## To choose the models for hyper parameter tuning

In [10]:
models={
    # "Random Forest":RandomForestClassifier(),
    "Decision Tree":DecisionTreeClassifier(),
    # "Logistic Regression":LogisticRegression(), #NOT GOOD
    # "GradientBoost Classifier": GradientBoostingClassifier(), #NOT GOOD
    # "Light GBM":LGBMClassifier(),
    # "XGB Classifier":XGBClassifier(),
    # "LinearSVC":LinearSVC()

}

for i in range(len(list(models))):

    model=list(models.values())[i]
    model.fit(X_train, y_train)

    #Make a prediction
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #Training set performance
    model_train_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred,average='weighted')
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_roc_auc_score=roc_auc_score(y_train,y_train_pred)

    #Test set performance
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred,average='weighted')
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_roc_auc_score=roc_auc_score(y_test,y_test_pred)

    # print(model)
    print('Model performance for Training set')
    print(f"- Accuracy: {model_train_accuracy}")
    print(f"- F1 Score: {model_train_f1}")
    print(f"- Precision Score: {model_train_precision}")
    print(f"- Recall Score: {model_train_recall}")
    print(f"- Roc Auc Score: {model_train_roc_auc_score}")

    print("--------------")
    print('Model performance for Test set')
    print(f"- Accuracy: {model_test_accuracy}")
    print(f"- F1 Score: {model_test_f1}")
    print(f"- Precision Score: {model_test_precision}")
    print(f"- Recall Score: {model_test_recall}")
    print(f"- Roc Auc Score: {model_test_roc_auc_score}")
    print("----------------------------------------")


# randomforest and decisiontrees will be option for further hyperparameter tuning

Model performance for Training set
- Accuracy: 1.0
- F1 Score: 1.0
- Precision Score: 1.0
- Recall Score: 1.0
- Roc Auc Score: 1.0
--------------
Model performance for Test set
- Accuracy: 0.6601899827288429
- F1 Score: 0.6621287848993053
- Precision Score: 0.43646408839779005
- Recall Score: 0.4546762589928058
- Roc Auc Score: 0.6014898876703696
----------------------------------------


In [ ]:
#Hyperparameter training
rf_params={"max_depth":[5,8,15,12,10],
           "max_features":[5,7,1,2,3,4,9,12,20,30,40],
           "min_samples_split":[2,8,15,20,30,40],
           "n_estimators":[100,150,200,300]
           }

#best param {'n_estimators': 50, 'max_depth': 20, 'learning_rate': 0.1}
# bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
xgboost_params={"max_depth":[10,20,30,40],
        #    "max_features":[5,7,8,9],
           "learning_rate":[0.001,0.01,0.5,0.1,1],
           "n_estimators":[32,50,100,200]
           }
#models list for hyperparameter tuning

randomcv_models=[
    ("RF",RandomForestClassifier(), rf_params),
    # ("XGBoost",XGBClassifier(), xgboost_params),

    # ("DT",DecisionTreeClassifier(), dt_params),
]


In [ ]:
model_param={}

for name, model, params in randomcv_models:
    random=RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for model_name in model_param:
    print(f"best param for {model_name}-----")
    print(model_param[model_name])

In [ ]:
# https://www.datacamp.com/tutorial/random-forests-classifier-python
from scipy.stats import randint
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)